# Stochastic Solver Experimentation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
x = "500670084407000630068000701000000209050980460679042518105869047796400852003720096"
y = "532671984417298635968534721384156279251987463679342518125869347796413852843725196"

dim = 9
shp = (dim, dim)
values = np.asarray(list(x), dtype=np.uint8).reshape(shp)
frozen = values.copy()
values, frozen

In [ ]:
def setup_probability_matrix(shp: tuple, frozen: np.ndarray) -> np.ndarray:
    probs: np.ndarray = np.ones(shp, dtype=np.float32)
    probs[frozen != 0] = 0.0
    return probs

In [ ]:
def check_row_hard(i: int, j: int, d: int, v: int, frozen: np.ndarray) -> bool:
    return v not in frozen[i, :]


def check_col_hard(i: int, j: int, d: int, v: int, frozen: np.ndarray) -> bool:
    return v not in frozen[:, j]


def check_box_hard(i: int, j: int, d: int, v: int, frozen: np.ndarray) -> bool:
    return v not in frozen[(i // d) : (i // d) + d, (j // d) : (j // d) + d]


def check_row_soft(i: int, j: int, d: int, values: np.ndarray) -> bool:
    return dict(zip(*np.unique(values[i, :], return_counts=True)))[values[i, j]] == 1


def check_col_soft(i: int, j: int, d: int, values: np.ndarray) -> bool:
    return dict(zip(*np.unique(values[:, j], return_counts=True)))[values[i, j]] == 1


def check_box_soft(i: int, j: int, d: int, values: np.ndarray) -> bool:
    v: int = values[i, j]
    s: int = (i // d) * d
    k: int = (j // d) * d
    return dict(zip(*np.unique(values[s : s + d, k : k + d], return_counts=1)))[v] == 1

In [ ]:
def update_probability_matrix(probs: np.ndarray, guess: np.ndarray, frozen: np.ndarray):
    d: int = int(np.sqrt(probs.shape[0]))
    for (i, j), v in np.ndenumerate(guess):
        if probs[i, j] == 0:
            continue
        if not check_row_hard(i, j, d, guess[i, j], frozen):
            probs[i, j] = 1.0
        elif not check_col_hard(i, j, d, guess[i, j], frozen):
            probs[i, j] = 1.0
        elif not check_box_hard(i, j, d, guess[i, j], frozen):
            probs[i, j] = 1.0
        else:
            probs[i, j] = 0.000001
            probs[i, j] += 0.1 if not check_row_soft(i, j, d, guess) else 0
            probs[i, j] += 0.1 if not check_col_soft(i, j, d, guess) else 0
            probs[i, j] += 0.1 if not check_box_soft(i, j, d, guess) else 0

In [ ]:
def generate_guess(values: np.ndarray, probs: np.ndarray, shp: tuple) -> np.ndarray:
    n: int = values.size
    x = np.stack((values.reshape(-1), np.random.randint(1, dim, (n,))), axis=1)
    p = probs.reshape(-1)
    odds = np.stack((np.abs(p - 1), p), axis=1)
    guess = np.zeros((n,))
    for i, x in enumerate(x, 0):
        guess[i] = np.random.choice(x, p=odds[i])
    return guess.reshape(shp)

In [ ]:
def solve(values, frozen, shp):
    probs: np.ndarray = setup_probability_matrix(shp, frozen)
    state: np.ndarray = generate_guess(values, probs, shp)
    for step in range(1000):
        print("Step:", step)
        # temp: float = 1 - (step + 1) / 1000
        p0 = np.sum(probs == 1)
        iterstop = 0
        while np.sum(probs == 1) >= p0:
            state = generate_guess(state, probs, shp)
            update_probability_matrix(probs, state, frozen)
            iterstop += 1
            if iterstop > 100:
                break
        p0 = np.sum(probs == 1)
        print(p0)
    return state


# Let s = s0
# For k = 0 through kmax (exclusive):
# T ← temperature( 1 - (k+1)/kmax )
# Pick a random neighbour, snew ← neighbour(s)
# If P(E(s), E(snew), T) ≥ random(0, 1):
# s ← snew
# Output: the final state s

solve(values, frozen, shp)

#### Visualising Temperature Function

In [ ]:
from matplotlib.axes import Axes

In [ ]:
kmax: int = 100000
temp: list[float] = []
for k in range(kmax):
    temp.append(1 - (k + 1) / kmax)
plot: Axes = sns.lineplot(temp)
plot.set_xlabel("Step Number")
plot.set_ylabel("Temperature")
plt.show()